In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
# Read CSV file
data = pd.read_csv('melb_data.csv')

# Drop useless features
data.drop(['Address', 'Lattitude', 'Longtitude', 'Propertycount', 'Date', 'Postcode', 'Method'], axis=1, inplace=True)

categorical_cols = ['Suburb', 'Type', 'SellerG', 'Regionname', 'CouncilArea', 'Rooms', 'YearBuilt']
# Replace categorical NA with -
for col in data.columns:
    if col in categorical_cols:
        if col == 'YearBuilt':
            data[col].fillna(0, inplace=True)
        else:
            data[col].fillna('-', inplace=True)
# Replace numerical NA with mean
for col in data.columns:
    if col not in categorical_cols:
        data[col].fillna(data[col].mean(), inplace=True)

data.head()

,Suburb,Rooms,Type,Price,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname
0,Abbotsford,2,h,1480000.0,Biggin,2.5,2.0,1.0,1.0,202.0,151.96765,0.0,Yarra,Northern Metropolitan
1,Abbotsford,2,h,1035000.0,Biggin,2.5,2.0,1.0,0.0,156.0,79.00000,1900.0,Yarra,Northern Metropolitan
2,Abbotsford,3,h,1465000.0,Biggin,2.5,3.0,2.0,0.0,134.0,150.00000,1900.0,Yarra,Northern Metropolitan
3,Abbotsford,3,h,850000.0,Biggin,2.5,3.0,2.0,1.0,94.0,151.96765,0.0,Yarra,Northern Metropolitan
4,Abbotsford,4,h,1600000.0,Nelson,2.5,3.0,1.0,2.0,120.0,142.00000,2014.0,Yarra,Northern Metropolitan


In [3]:
mappings = {}
# Convert categorical features to numeric
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    data[col] = le.transform(data[col])
data

# Pickle mappings
import pickle
with open('mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)


In [4]:
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(data.drop('Price', axis=1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['Price'], test_size=0.2, random_state=42)
print ("X_train.shape = ", X_train.shape)
print ("y_train.shape = ", y_train.shape)
print ("X_val.shape = ", X_test.shape)
print ("y_val.shape = ", y_test.shape)

X_train.shape =  (10864, 13)
y_train.shape =  (10864,)
X_val.shape =  (2716, 13)
y_val.shape =  (2716,)


In [5]:
def custom_loss_function(y_true, y_pred):    
    y_diff = tf.abs(tf.math.log(1+y_true) - tf.math.log(1+y_pred))    
    return tf.reduce_sum(tf.square(y_diff)) / tf.cast(tf.size(y_diff), tf.float32)

In [6]:
model2 = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model2.compile(optimizer='adam', loss=custom_loss_function, metrics=['mae'])

In [7]:
model2.fit(X_train, y_train, epochs=10, batch_size=2, verbose=1, validation_data=(X_test,y_test))

Epoch 1/10
5432/5432 [==============================] - 12s 2ms/step - loss: 19.5974 - mae: 977816.0625 - val_loss: 1.1576 - val_mae: 716877.5625
Epoch 2/10
5432/5432 [==============================] - 12s 2ms/step - loss: 0.3535 - mae: 441681.3750 - val_loss: 0.1375 - val_mae: 314057.7500
Epoch 3/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.1453 - mae: 323885.0938 - val_loss: 0.1131 - val_mae: 293183.3750
Epoch 4/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.1314 - mae: 315143.2188 - val_loss: 0.1083 - val_mae: 287413.9375
Epoch 5/10
5432/5432 [==============================] - 11s 2ms/step - loss: 0.1269 - mae: 310168.8750 - val_loss: 0.1056 - val_mae: 285117.4062
Epoch 6/10
5432/5432 [==============================] - 10s 2ms/step - loss: 0.1227 - mae: 304499.4375 - val_loss: 0.1042 - val_mae: 283145.4375
Epoch 7/10
5432/5432 [==============================] - 10s 2ms/step - loss: 0.1216 - mae: 304609.9375 - val_loss: 0.1018 - val_m

In [8]:
y_pred = model2.predict(X_test)
print(y_test.to_numpy(), y_pred)

85/85 [==============================] - 0s 816us/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [[1503428.6 ]
 [ 587361.4 ]
 [ 682262.44]
 ...
 [1896631.2 ]
 [1725706.4 ]
 [ 926538.75]]


In [9]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(50,activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [10]:
model.compile(optimizer='adam',
              loss=custom_loss_function)
history = model.fit(X_train,y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(X_test,y_test))

Epoch 1/50
340/340 [==============================] - 1s 3ms/step - loss: 91.8683 - val_loss: 53.6969
Epoch 2/50
340/340 [==============================] - 1s 3ms/step - loss: 42.4723 - val_loss: 34.0400
Epoch 3/50
340/340 [==============================] - 1s 2ms/step - loss: 28.9433 - val_loss: 24.4992
Epoch 4/50
340/340 [==============================] - 1s 3ms/step - loss: 21.3699 - val_loss: 18.5163
Epoch 5/50
340/340 [==============================] - 1s 3ms/step - loss: 16.3881 - val_loss: 14.3442
Epoch 6/50
340/340 [==============================] - 1s 3ms/step - loss: 12.7863 - val_loss: 11.2665
Epoch 7/50
340/340 [==============================] - 1s 3ms/step - loss: 10.0892 - val_loss: 8.9146
Epoch 8/50
340/340 [==============================] - 1s 2ms/step - loss: 8.0087 - val_loss: 7.0749
Epoch 9/50
340/340 [==============================] - 1s 2ms/step - loss: 6.3725 - val_loss: 5.6142
Epoch 10/50
340/340 [==============================] - 1s 2ms/step - loss: 5.0512 - val

In [11]:
y_pred = model.predict(X_test).flatten()
print(y_test.to_numpy(), y_pred)

# calculate MAE of the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

85/85 [==============================] - 0s 798us/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [1688972.2   551336.8   754508.9  ... 2083097.8  1408451.8   869220.94]
MAE: 242028.074


In [12]:
model.save('Model1.h5')
model2.save('Model2.h5')